<a href="https://colab.research.google.com/github/jipperspippers/drone_tracking/blob/main/YOLOV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Data and Packages


In [ ]:
#implementing some YOLOv8 model similar to the one shown below, would be optimal for use with our LSTM
# the following is a proposed but unimplmnted model that would work best for the dataset, --> a indepth look at ultralytics YOLOv8 and other yolo models
# that we base our lstm and image detection on
!pip3 install -U -q PyDrive
!pip3 install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import patoolib
import pandas as pd
import os
import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded_file_name = '/content/drive/MyDrive/MASTER_DATASET/Training_data_V_MASTER.rar'
extracted_folder_name = 'dataset'
file_id = '1fEmfgDrGI-HxF5V1s8wNko9fAo5F62Wp'

# Create a GoogleDriveFile instance with the file ID
file = drive.CreateFile({'id': file_id})
file.GetContentFile(downloaded_file_name)  # Replace 'downloaded_file.ext' with the desired file name


# Create the folder if it doesn't exist
os.makedirs(extracted_folder_name, exist_ok=True)

# Extract the contents of the RAR file
patoolib.extract_archive(downloaded_file_name, outdir=extracted_folder_name)

print(f"Contents of {downloaded_file_name} extracted to {extracted_folder_name}")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!mv /content/drive/MyDrive/MASTER_DATASET/Training_data_V_MASTER.csv /content/dataset/
!mkdir /content/dataset/labels

mv: cannot stat '/content/drive/MyDrive/MASTER_DATASET/Training_data_V_MASTER.csv': No such file or directory


# The Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.ops as ops
from torchvision import datasets, transforms
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd


In [ ]:
# TO-DO
'''
- check what it means to have strides in SPPF
- implement widen factors into the channels
- implement ratios into the channels
- finish up the RNN (generate 60frames as outputs)
'''

class ConvBatchSiLu(nn.Module):
  def __init__(self, in_channels, out_channels, kernel, stride, padding):
    super(ConvBatchSiLu, self).__init__()
    #print(in_channels, out_channels, kernel, stride, padding)
    self.output = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel, stride, padding, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.SiLU(inplace=True)
    )

  def forward(self, X):
    return self.output(X)

class Detect(nn.Module):
  def __init__(self, in_channels, out_channels, nc, reg_max):
    """
    in_channels: int
    out_channels: int
    nc: int, number of classes
    reg_max: int
    """
    super(Detect, self).__init__()
    out_channels = round(out_channels)
    self.convBatch = ConvBatchSiLu(in_channels, out_channels, 3, 1, 1)
    self.convBatch2 = ConvBatchSiLu(out_channels, out_channels, 3, 1, 1)
    self.conv1 = nn.Conv2d(out_channels, round(4*reg_max), 1, 1)
    self.conv2 = nn.Conv2d(out_channels, round(nc), 1, 1)
    self.sig = nn.Sigmoid()
  def forward(self, X):
    c1 = self.convBatch2(self.convBatch(X))

    return self.conv1(c1), self.sig(self.conv2(c1))

class BottleNeck(nn.Module):
  def __init__(self, shortcut, in_channels, out_channels):
    super(BottleNeck, self).__init__()
    self.shortcut = shortcut
    self.conv1 = ConvBatchSiLu(in_channels, round(in_channels/2), 3, 1, 1)
    self.conv2 = ConvBatchSiLu(round(in_channels/2), out_channels, 3,1,1)

  def forward(self, X):
    if self.shortcut:
      return self.conv2(self.conv1(X)) + X
    else:
      return self.conv2(self.conv1(X))



class SPPF(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super(SPPF, self).__init__()
    c_ = round(in_channels//2)
    self.conv1 = ConvBatchSiLu(in_channels, c_, 1,1,0)
    self.pool = nn.MaxPool2d(5,1,2)
    self.conv2 = ConvBatchSiLu(c_*4, out_channels, 1, 1, 0)
  def forward(self, X):
    c1 = self.conv1(X)
    pool1 = self.pool(c1)
    pool2 = self.pool(pool1)
    pool3 = self.pool (pool2)
    concatted = torch.cat((c1, pool1, pool2, pool3), dim=1)
    return self.conv2(concatted)

class C2f(nn.Module):
  def __init__(self,  in_channels, out_channels ,shortcut, n):
    super(C2f, self).__init__()
    self.n = n
    self.shortcut = shortcut
    self.c_out = out_channels
    self.conv = ConvBatchSiLu(round(in_channels), round(out_channels), 1, 1, 0)
    self.bottleneck = BottleNeck(shortcut, round((out_channels)/2), round(out_channels/2))
    self.c_in = round(out_channels * 0.5 * (n+2) )
    self.conv2 = ConvBatchSiLu(round(self.c_in), round(out_channels), 1,1,0) #figure out how much in


  def forward(self, X):
    c1 = self.conv(X)
    split = torch.split(c1, round(c1.shape[1]/2), dim = 1) # hopefully this modifies the channels
    #print("c_in: ", self.c_in, "c_out:", self.c_out, "n:", self.n, "X shape" ,X.shape, "split0:", split[0].shape, "split1", split[1].shape)


    b = self.bottleneck(split[0])
    k = 1
    out = b
    for i in range(1, self.n):
      b = self.bottleneck(b)
      out = torch.cat((out, b), dim=1)
      k += 1

    out = torch.cat((c1, out), dim=1)

    #print("out: ", out.shape)
    return self.conv2(out)

class YOLO(nn.Module):
  def __init__(self, in_channels, reg_max=1.0, deepen_factor=1.0, widen_factor=1.0, ratio=1.0, nc=1):
    """
    in_channels: int
    reg_max: double,
    deepen_factor: double
    widen_factor: double
    ratio: double
    nc: int
    """
    super(YOLO, self).__init__()
    base_channels = 64
    # Create the backbone
    self.convBackbone0 = ConvBatchSiLu(in_channels, round(base_channels*widen_factor), 3, 2, 1)
    self.convBackbone1 = ConvBatchSiLu(round(base_channels*widen_factor), round(base_channels*widen_factor*2), 3, 2, 1)
    self.convBackbone2 = ConvBatchSiLu(round(base_channels*widen_factor*2), round(base_channels*widen_factor*4), 3, 2, 1)
    self.convBackbone3 = ConvBatchSiLu(round(base_channels*widen_factor*4), round(base_channels*widen_factor*8), 3, 2, 1)
    self.convBackbone4 = ConvBatchSiLu(round(base_channels*widen_factor*8), round(base_channels*8*widen_factor*ratio), 3, 2, 1)
    self.SPPF = SPPF(round(base_channels*8*widen_factor*ratio), round(base_channels*8*widen_factor*ratio), stride=32)


    # C2f Backbones
    self.c2fBackbone1 = C2f(base_channels*2*widen_factor, base_channels*2*widen_factor, True, round(3*deepen_factor))
    self.c2fBackbone2 = C2f(base_channels*4*widen_factor, base_channels*4*widen_factor, True, round(6*deepen_factor) )
    self.c2fBackbone3 = C2f(base_channels*8*widen_factor, base_channels*8*widen_factor, True, round(6*deepen_factor) )

    self.c2fBackbone4 = C2f(round(base_channels*8*widen_factor*ratio), round(base_channels*8*widen_factor*ratio), True, round(3*deepen_factor))

    #C2f Backbone connnection to neck
    self.c2fBackbone5 = C2f(base_channels*4*widen_factor, base_channels*4*widen_factor,True,  round(3*deepen_factor))
    self.c2fBackbone6 = C2f(base_channels*8*widen_factor, base_channels*8*widen_factor,True, round(6*deepen_factor))

    #Neck and SPPF
    #self.c2fNeck1 = C2f(base_channels*8*(1+ratio)*widen_factor,  base_channels*8*widen_factor, False, round(3*deepen_factor))
    self.c2fNeck1 = C2f(base_channels*8*(1+ratio)*widen_factor,  base_channels*4*widen_factor, False, round(3*deepen_factor))
    self.c2fNeck2 = C2f(base_channels*8*widen_factor,  base_channels*4*widen_factor, False, round(3*deepen_factor))
    # self.c2fNeck3 = C2f(base_channels*8*widen_factor, base_channels*8*widen_factor, False, round(3*deepen_factor) )
    # self.c2fNeck4 = C2f(base_channels*8*widen_factor*(1+ratio), base_channels*8*widen_factor, False, round(3*deepen_factor) )

    # self.convNeck1 = ConvBatchSiLu( round(base_channels*4*widen_factor),  round(base_channels*4*widen_factor), 3, 2, 1)
    # self.convNeck2 = ConvBatchSiLu( round(base_channels*8*widen_factor), round(base_channels*8*widen_factor), 3, 2, 1)

    # upsampling the width and height
    self.upsample = nn.Upsample(scale_factor = 2)

    #Detect heads
    self.detect1 = Detect( round(base_channels*4*widen_factor),  round(base_channels*4*widen_factor),nc, reg_max)

  def forward(self, X):
    #print("Before: \n")
    p1 =  self.convBackbone0(X)
    # Backbone
    #print("Backbone: \n")
    p2 = self.convBackbone1(p1)
    c2f_backbone1 = self.c2fBackbone1(p2)
    p3 = self.convBackbone2(c2f_backbone1)
    c2f_backbone2 = self.c2fBackbone2(p3)
    p4 = self.convBackbone3(c2f_backbone2)
    c2f_backbone3 = self.c2fBackbone3(p4)
    p5 = self.convBackbone4(c2f_backbone3)
    c2f_backbone4 = self.c2fBackbone4(p5)
    sppf = self.SPPF(c2f_backbone4)
    #print("Backbone: Done \n")

    #print("Backbone connection: \n")

    # the connection between neck and backbone
    c2f_backbone5 = self.c2fBackbone5(p3)
    c2f_backbone6 = self.c2fBackbone6(p4)
    #print("Backbone connection: Done \n")

    # the neck
    #print("Neck \n")
    concat1 = torch.cat((self.upsample(sppf),c2f_backbone6), dim=1)
    c2f_neck1 = self.c2fNeck1(concat1)
    concat2 = torch.cat((self.upsample(c2f_neck1), c2f_backbone5), dim=1)
    c2f_neck2 = self.c2fNeck2(concat2)

    # c1 = self.convNeck1(c2f_neck2)
    # c2f_neck3 = self.c2fNeck3(torch.cat((c2f_neck1, c1), dim=1))
    # c2 = self.convNeck2(c2f_neck3)
    # c2f_neck4 = self.c2fNeck4(torch.cat((sppf, c2), dim=1))

    #print("Neck Done \n")

    # the head
    #print("Detect:")
    head1 = self.detect1(c2f_neck2)
    #print("Detect Done")
    return head1


In [ ]:
from torchsummary import summary
model = YOLO(in_channels=3, nc=4, widen_factor=0.5,deepen_factor=0.33)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(summary(model, (3,416,416)))

# first is the bounding box prediction in (xmin, ymin, xmax, ymax)
# based on the position of the
# second are the classes and objects detected.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 208, 208]             864
       BatchNorm2d-2         [-1, 32, 208, 208]              64
              SiLU-3         [-1, 32, 208, 208]               0
     ConvBatchSiLu-4         [-1, 32, 208, 208]               0
            Conv2d-5         [-1, 64, 104, 104]          18,432
       BatchNorm2d-6         [-1, 64, 104, 104]             128
              SiLU-7         [-1, 64, 104, 104]               0
     ConvBatchSiLu-8         [-1, 64, 104, 104]               0
            Conv2d-9         [-1, 64, 104, 104]           4,096
      BatchNorm2d-10         [-1, 64, 104, 104]             128
             SiLU-11         [-1, 64, 104, 104]               0
    ConvBatchSiLu-12         [-1, 64, 104, 104]               0
           Conv2d-13         [-1, 16, 104, 104]           4,608
      BatchNorm2d-14         [-1, 16, 1

# Training

In [ ]:
import torch
import pandas as pd
from torchvision import datasets, transforms
torch.manual_seed(42)


# TODO : change this to which we organize the data

def process():
  '''
  # Each file will look like something like this, where the first column is
  # the type of thing
  # 0 x y w h
  # 0 x y w h
  # 1 x y w h
  '''
  # create a new file for each row
  # in each file it is
  df = pd.read_csv('/content/dataset/Training_data_V_MASTER.csv')
  df.fillna('', inplace=True)

  # create a new file with format
  # category x y w h
  # on each row.
  #print(df)
  for i, row in df.iterrows():
      file_name = row['imageFilename'].replace('Training_data_V_MASTER\\', '')

      plane = row.filter(regex='AIRPLANE', axis=0).values
      heli = row.filter(regex='HELICOPTER', axis=0).values
      drone = row.filter(regex='DRONE', axis=0).values
      bird = row.filter(regex='BIRD', axis=0).values

      # Should i make a folder for each?
      # like if i were to iterate through each folders
      file_path = f"/content/dataset/labels/{file_name}.txt"

      with open(file_path, 'w') as f:
          for i in range(0, len(plane), 4):
              f.write('0')
              f.write(f' {plane[i]} {plane[i+1]} {plane[i+2]} {plane[i+3]}\n')

          for i in range(0, len(drone), 4):
              f.write('1')
              f.write(f' {drone[i]} {drone[i+1]} {drone[i+2]} {drone[i+3]}\n')

          for i in range(0, len(heli), 4):
              f.write('2')
              f.write(f' {heli[i]} {heli[i+1]} {heli[i+2]} {heli[i+3]}\n')

          for i in range(0, len(bird), 4):
              f.write('3')
              f.write(f' {bird[i]} {bird[i+1]} {bird[i+2]} {bird[i+3]}\n')




# def get_data(batch_size):
#     # Load image data
#     transform = transforms.Compose([transforms.ToTensor()])
#     dataset = datasets.ImageFolder('/content/train', transform=transform)
#     csv_file_path = '/content/drive/MyDrive/0/master_data.csv'
#     data_frame = pd.read_csv(csv_file_path)

#     # Convert to PyTorch tensor
#     data_frame_numeric = data_frame.apply(pd.to_numeric, errors='coerce')
#     csv_dataset = torch.tensor(data_frame_numeric.values, dtype=torch.float32)


#     # Split the data into training and validation sets
#     train_size = int(0.8 * len(dataset ))
#     valid_size = len(dataset ) - train_size

#     train_csv, valid_csv = torch.utils.data.random_split(csv_dataset, [train_size, valid_size])
#     train_data, valid_data = torch.utils.data.random_split(dataset, [train_size, valid_size])

#     # Create DataLoader instances
#     train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
#     valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)

#     return train_csv, valid_csv, train_loader, valid_loader

process()

batch_size = 32
#train_csv, valid_csv, train_loader, valid_loader = get_data(batch_size)


                                           imageFilename  AIRPLANE_1  \
0         Training_data_V_MASTER\V_AIRPLANE_0011_001.png       313.0   
1         Training_data_V_MASTER\V_AIRPLANE_0011_002.png  313.448181   
2         Training_data_V_MASTER\V_AIRPLANE_0011_003.png   311.48938   
3         Training_data_V_MASTER\V_AIRPLANE_0011_004.png  309.946655   
4         Training_data_V_MASTER\V_AIRPLANE_0011_005.png  308.959167   
...                                                  ...         ...   
86978  Training_data_V_MASTER\V_HELICOPTER_061278_303...               
86979  Training_data_V_MASTER\V_HELICOPTER_061278_304...               
86980  Training_data_V_MASTER\V_HELICOPTER_061278_305...               
86981  Training_data_V_MASTER\V_HELICOPTER_061278_306...               
86982  Training_data_V_MASTER\V_HELICOPTER_061278_307...               

       AIRPLANE_2 AIRPLANE_3 AIRPLANE_4 AIRPLANE_5 AIRPLANE_6 AIRPLANE_7  \
0           205.0       36.0       24.0                    

In [ ]:
# Rework the data processing first


def getClassBox(pred, conf, max_boxes=1):
  num_preds, nc, w, h = pred[1].shape
  classes = torch.empty((1, w*h))

  for i in range(0, num_preds):
    vec = pred[1][i]
    flat = torch.flatten(vec, 1, 2)
    # check each entrance and see what is the max value of the grid
    for j in range(0, w*h):
      classes[j] = torch.max(flat, dim=0)


  return classes

def getBboxes(pred, height, width):
  """
  pred: tuple (tuple( Tensor of size n x c x height x width  , Tensor of size n x c x height x width)) of len 3
  returns: Tensor of size n x 4, (n, nc, s^2 (Xc, Yc, width, height)
  """
  num_preds, c, w, h = pred[0].shape
  _, nc, _, _ = pred[1].shape


  classes  = getClassBox(pred, conf, max_boxes)
  bbox = torch.empty((num_preds,4,max_boxes))
  values = torch.empty((num_preds,1))
  pred_class = torch.empty((num_preds,1))


  # 0 , 1 ,2 are each different sizes
  for i in range(0, num_preds):
    vec = pred[0][i]
    flat = torch.flatten(vec, 1, 2)
    # for each box we
    for j in range(0, c):
      bbox[i][j] = flat[j]

  return bbox, classes


def xywh_to_xyxy(bbox):
  """
  bbox: Tensor of size n x 4
  returns: Tensor of size n x 4
  """
  bbox[:,2] += bbox[:,0]
  bbox[:,3] += bbox[:,1]
  return bbox


In [ ]:
out = model.forward(torch.randn(1,3,416,416))
print(out[1].shape)

torch.Size([1, 4, 52, 52])


In [ ]:
bounding_boxes, pred_class =  getBboxes(out, 416, 416)
print( pred_class)


ValueError: ignored

In [ ]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for data_in, labels in data_loader:
        output = model(data_in)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [ ]:
bloss = ops.generalized_box_iou_loss # box position loss
closs = nn.functional.binary_cross_entropy # class loss

def train_model(model, train, valid,train_csv, valid_csv,bloss, closs, height, width, num_classes, max_boxes = 1, num_epochs=2, learning_rate=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses, train_acc, valid_acc = [], [], []
    model.train()
    model.requires_grad_()
    epochs = []

    for epoch in range(num_epochs):
      for idx, image in enumerate(train):
          label = train_csv[0:batch_size] # label, x, y, w, h
          optimizer.zero_grad()
          pred = model(image[0])
          pred_box, pred_class = getBboxes(pred, height, width) # we get boxes here


          # get the best bounding box
          for j in range(0, pred_box.shape[0]):
              for i in range(0, max_boxes):
                  temp_losses = []
                  for k in range(0, num_classes):
                      # Assuming that box labels are in form xc,yc, width, height

                      x =  pred_box[j, k, 0,i]
                      y = pred_box[j, k, 1,i]
                      w = pred_box[j, k, 0,i] + pred_box[j, k,2,i]
                      h =  pred_box[j, k,1,i] + pred_box[j, k, 3,i]

                      # Create a new tensor with extracted numeric values
                      b = torch.tensor([x, y, w, h], dtype=torch.double)
                      box = torch.tensor([label[i][1], label[i][2], label[i][1]+label[i][3], label[i][2]+label[i][4]]).double()
                      bounding_loss = bloss( b, box)
                      bounding_loss.requires_grad=True
                      temp_losses.append(bounding_loss)


          print(label[0], pred_class)
          class_loss = closs(torch.mean(pred_class))
          class_loss.requires_grad = True

          loss = min(temp_losses) + class_loss
          loss.requires_grad = True

          loss.backward()
          optimizer.step()
          losses.append(float(loss))




# outputs of this model => bounding boxes
# get bounding boxes => from function
# compare bounding boxes w/ ground truth boudning boxes
# bounding boxes (xc, yc, width, height)
# (xc , yc, width, height)

In [ ]:
train_loader.dataset[1][0].shape
train_csv.dataset

In [ ]:

height = train_loader.dataset[1][0].shape[1]
width = train_loader.dataset[1][0].shape[2]
num_classes = 4
max_boxes = 1
print(width,height)
train_model(model, train_loader, valid_loader, train_csv, valid_csv, bloss,
            closs, width, height,num_classes, max_boxes)


# TO GET THE BOUNDING BOXES:
# b <= max_boxes
# c <= num_classes
# i = batch_index, c = class , b = index of a box
# x=model(input)[i][c][0][b]
# y=model(input)[i][c][1][b]
# w=model(input)[i][c][2][b]
# h=model(input)[i][c][3][b]